In [5]:
import os
import sys
from glob import glob
import json

In [7]:
base = "../../Report-nmi-wsi"
path = "test_annotation.json"
file_path = os.path.join(base,path)
print(file_path)

../../Report-nmi-wsi/test_annotation.json


In [29]:
# Read the JSON file
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Extract the keys and print them
keys = list(data.keys())
# print("Keys:", keys)

# Print the first key-value pair
if keys:
    first_key = keys[0]
    first_value = data[first_key]['caption']
    print("First Key:", first_key)
    print("First Value:", first_value)
else:
    print("The JSON file is empty.")

First Key: 1_00143_sub0_021
First Value: ['Minimal variability in nuclear size shape and outline consistent with moderate pleomorphism. Mild nuclear crowding is seen. There is marked disorganization and lack of cellular polarity toward the surface urothelium. Mitosis is infrequent. The nucleoli of nuclei are prominent. High grade.', 'Minimal variability in nuclear size shape and outline consistent with moderate pleomorphism. There is a mild degree of crowding. Architecturally the cells show complete lack of polarity toward the surface urothelium. There are infrequent mitotic figures throughout the tissue. Prominent nucleoli are easily identified in low magnification scanning. High grade.', 'Moderate pleomorphism and cytologic atypia is present. Nuclei are mildly crowded together. Polarity is completely lost. Mitosis is infrequent throughout the tissue. Prominent nucleoli are easily identified in low magnification scanning. High grade.', 'Moderate pleomorphism is present. The nuclei are

In [11]:
filename = "fadfa.bin"
base_name, _ = os.path.splitext(filename)
new_filename = base_name + ".jpg"
print(new_filename)

fadfa.jpg


In [12]:
!ls graph

assignment_mat cell_graphs    tissue_graphs


In [16]:
cg_path = "graph/cell_graphs/test"
cg = glob(os.path.join(cg_path, '*.bin'))
cg.sort()
num_cg = len(cg)
print(cg)

['graph/cell_graphs/test/1_00061_sub0_002.bin', 'graph/cell_graphs/test/1_00061_sub0_004.bin']


In [34]:
new_file_names = [os.path.splitext(os.path.split(i)[-1])[0] for i in cg]
print(new_file_names)

['1_00061_sub0_002', '1_00061_sub0_004']


In [21]:
for i in cg:
    spt = os.path.split(i)[-1]
    base_name, _ = os.path.splitext(spt)
    new_filename = base_name + ".jpg"
    print(new_filename)

1_00061_sub0_002.jpg
1_00061_sub0_004.jpg


In [53]:
!ls ../../Report-nmi-wsi

Images                nmi-wsi.zip           train_annotation.json
Images.zip            test
class.json            test_annotation.json


In [51]:
all_image = glob("../../Report-nmi-wsi/Images/*.png")
print(len(all_image))

4253


In [54]:
test_report = "../../Report-nmi-wsi/test_annotation.json"
train_report = "../../Report-nmi-wsi/train_annotation.json"

with open(test_report, 'r') as test_file:
    t_data = json.load(test_file)
    print(len(t_data.keys()))

with open(train_report, 'r') as train_file:
    tr_data = json.load(train_file)
    print(len(tr_data.keys()))



1889
2364


In [56]:
a = '1_00061_sub0_002'
if a in t_data.keys():
    print(a)
if a in tr_data.keys():
    print("fdsfas")

fdsfas


In [69]:
# report_path = "../../Report-nmi-wsi"
# split = "train"
# split_annotation = split+"_annotation.json"
# report_path = os.path.join(report_path,split_annotation)
# print(report_path)
# print(file_path)
# with open(report_path, 'r') as json_file:
#     report_data = json.load(json_file)
#     print(report_data)
# print(new_file_names)
# values_in_order = [report_data[key]['caption'] for key in new_file_names if key in report_data.keys()]
# print(len(values_in_order[0]))

In [16]:
from dgl.data.utils import load_graphs
from glob import glob
graph_path = "./graph"
split = "test" # Train Test Eval
cg_path = os.path.join(graph_path,"cell_graphs",split)
print(cg_path)

cgs = glob(os.path.join(cg_path,'*bin'))
print(cgs)
print()
cell_graphs = [load_graphs(cg) for cg in cgs]
for i in cell_graphs:
    print(i[0][0[]])


./graph/cell_graphs/test
['./graph/cell_graphs/test/1_00061_sub0_004.bin', './graph/cell_graphs/test/1_00061_sub0_002.bin']

[Graph(num_nodes=290, num_edges=1419,
      ndata_schemes={'feat': Scheme(shape=(514,), dtype=torch.float32), 'centroid': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={})]
[Graph(num_nodes=387, num_edges=1908,
      ndata_schemes={'feat': Scheme(shape=(514,), dtype=torch.float32), 'centroid': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={})]


In [14]:
print(first[0])
print(first[0][0])

[Graph(num_nodes=290, num_edges=1419,
      ndata_schemes={'feat': Scheme(shape=(514,), dtype=torch.float32), 'centroid': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={})]
Graph(num_nodes=290, num_edges=1419,
      ndata_schemes={'feat': Scheme(shape=(514,), dtype=torch.float32), 'centroid': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={})


In [5]:
!ls graph/cell_graphs/test

test
